<img src="https://www.ufpr.br/portalufpr/wp-content/uploads/2015/11/ufpr_25.jpg" alt="Drawing" width="100"/>

**Universidade Federal do Paraná**

Programa de Pós-Graduação em Ciências Geodésicas (PPGCG)

Disciplina: CGEO7009 - Desenvolvimento de Aplicações Geoespaciais

Docente: Profa Dra. Silvana P. Camboim

Discentes de Doutorado: Darlan M. Nunes | Fabíola Andrade Souza

Data: 17/12/2021

**Projeto ENERG!**

<img src="https://energi.eletrica.ufpr.br/static/images/navbar/logo-colorido.svg" alt="Drawing" width="100"/>

---

###**Bibliotecas**

In [ ]:
# Instalando as Bibliotecas necessárias
!pip install streamlit
!pip install ipykernel>=5.1.2
!pip install pydeck
!pip install pyngrok
!pip install streamlit-folium
!pip install geopandas
!pip install psycopg2-binary

###**Preparando o ambiente no Google Drive**

In [1]:
# Importando a biblioteca
from google.colab import drive

# Isso irá pedir sua autorização
drive.mount('/content/drive')

# Agora, seu Drive estará disponível em: /content/drive/My Drive

ModuleNotFoundError: No module named 'google.colab'

### **APP Streamlit - Projeto ENERG!**

In [1]:
%%writefile energi.py

# Biblioteca para conexão com o banco de dados
from configparser import ConfigParser
import psycopg2

# Biblioteca para os dados em geojson
import json

# Bibliotecas para a página do streamlit, o mapa e gráfico
import streamlit as st
from streamlit_folium import folium_static
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as pxi


# --- CONEXÃO COM O BANCO DE DADOS DA UFPR ---#

def config(filename='/content/drive/My Drive/db-labgeolivre_projenergi.ini', section='postgresql'):
  # create a parser
  parser = ConfigParser()
  # lendo o arquivo config file
  parser.read(filename)

  # get section, default to postgresql
  db = {}
  if parser.has_section(section):
    params = parser.items(section)
    for param in params:
      db[param[0]] = param[1]
    else:
      raise Exception('Section {0} not found in the {1} file'.format(section, filename))
    return db

conn = None
try:
  # lendo os parâmetros de conexão
  params = config()
  # conecta oa banco PostgreSQL server
  print('Conectando ao banco de dados PostgreSQL...')
  conn = psycopg2.connect(**params)
  # Cria cursor
  cur = conn.cursor()
  print ('Conexao realizada com sucesso')
  print (cur)
except (Exception, psycopg2.DatabaseError) as error:
  print ("Não foi possível conectar com o Banco de Dados!")
  
  
# --- CRIAÇÃO DO MAPA --- #
# Tamanho do mapa
width = 850     #750
height = 550     #450

m = folium.Map(width = width,
              height = height,
              location = [-25.5,-49.3],
                            zoom_start = 11,
                            tiles = 'Cartodb Positron',
                            attr = 'Data by ©contribuidores do OpenStreetMap (CC BY-SA 2.0)')

folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)


# --- ESTILOS --- #
# Função que retorna o estilo dos polígonos de bairros
def style_bairros(feature):
    return {'fillColor': '#F0E68C', 'fillOpacity': 0.05, 'color': '#808080', 'weight' : 1   
    }

# Função que retorna o estilo dos polígonos de edificação categorizados pelo local
def style_edif(feature):
	return {'fillColor': '#CD5C5C' if feature['properties']['local'] == 'Politécnico' 
	        else '#4682B4' if feature['properties']['local'] == 'Agrárias'
	        else '#228B22' if feature['properties']['local'] == 'Botânico'
	        else '#9932CC' if feature['properties']['local'] == 'Juvevê'
	        else '#FFD700' if feature['properties']['local'] == 'Campus Batel'
          else '#800000',
					'fillOpacity': 0.5, 'color': '#A9A9A9', 'weight' : 1 
	}

# Função que retorna o estilo dos polígonos selecionados
def style_sel(feature):
    return {'fillColor': '#0000FF', 'fillOpacity': 0.4, 'color': '#0000CD', 'weight' : 1   
    }

  
# -- FUNÇÕES --- #
# Funções que buscam a lista de nomes dos locais da UFPR e medidores a partir das tabelas no bd - para comboBox
def lista_local():
  lista_local = []
  cur.execute ("""SELECT local FROM edificacao group by local;""")
  fet_list = cur.fetchall()
  conn.commit()
  lista_local.append("") # Para iniciar sem campus selecionado
  for codlocal in fet_list:
    lista_local.append(codlocal[0])
  return lista_local

def lista_medidor():
  lista_medidor = []
  cur.execute ("""SELECT nome FROM medidor group by nome;""")
  fet_list = cur.fetchall()
  conn.commit()
  lista_medidor.append("") # Para iniciar sem medidor selecionado
  for nome in fet_list:
    lista_medidor.append(nome[0])
  return lista_medidor


# Função seleciona a geometria dos bairros e carrega no mapa com simbologia simples
def bairros():
    cur.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
    FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT id, nome) As l)) As properties FROM bairros As lg ) As f ) As fc;""")
    json_bairros = json.dumps(cur.fetchall())
    conn.commit()
    gj = folium.GeoJson(json_bairros[2:len(json_bairros)-2],
                     name = 'Bairros Curitiba',
                     #tooltip=folium.GeoJsonTooltip(fields=['nome']), # Desabilitado pq o nome do bairro tem no basemap
                     style_function = lambda feature: style_bairros(feature)).add_to(m)


# Função seleciona a geometria das edificações e carrega no mapa com simbologia categorizada
def edif():
  cur.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
  FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT gid, local, nomeedif) As l)) As properties FROM edificacao As lg ) As f ) As fc;""")
  json_edif = json.dumps(cur.fetchall())
  conn.commit()
  gj = folium.GeoJson(json_edif[2:len(json_edif)-2],
                      name = 'Edificações UFPR',
                      tooltip=folium.GeoJsonTooltip(fields=['nomeedif'], aliases=["Nome edificação: "],
                                                    ),
                      style_function = lambda feature: style_edif(feature)).add_to(m)


# Função seleciona a geometria dos medidores e carrega no mapa com simbologia simples de marcador
# Não vamos aplicar esta função, porque a que filtra por campus ficou mais interessante
def medidor():
  cur.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
  FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT id_medidor, nome) As l)) As properties FROM medidor As lg ) As f ) As fc;""")
  json_medidor = json.dumps(cur.fetchall())
  conn.commit()
  layer = folium.FeatureGroup(name='Medidores de energia')
  gj = folium.GeoJson(json_medidor[2:len(json_medidor)-2],
                      name = 'Medidores de energia',
                      )
  
  # iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
  for feature in gj.data['features']:
    if feature['geometry']['type'] == 'Point':
      folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
                    tooltip=folium.features.GeoJsonTooltip(fields=['nome'],
                                                           icon = folium.Icon(color = "lightgreen", icon = "star"),
      )).add_to(layer)
  layer.add_to(m)


# Função carrega no mapa o WMS dos medidores classificados por círculos concêntricos / O WMS não está aparecendo...
def med_wms():
  url = 'http://200.17.225.97/geoserver/teste/wms?service=WMS&version=1.1.0&request=GetMap&layers=teste%3Amedidor&bbox=-49.28330612182617%2C-25.457033157348633%2C-49.2309455871582%2C-25.41175651550293&width=768&height=664&srs=EPSG%3A4674&styles=&format=application/openlayers'
  w = folium.WmsTileLayer(url=url,
                         layers='medidor',
                         name='Medidores por Consumo',
                         format='image/png',
                         shown=True
                         )
  w.add_to(m)
#URL COMPLETA DO WMS MEDIDOR: http://200.17.225.97/geoserver/teste/wms?service=WMS&version=1.1.0&request=GetMap&layers=teste%3Amedidor&bbox=-49.28330612182617%2C-25.457033157348633%2C-49.2309455871582%2C-25.41175651550293&width=768&height=664&srs=EPSG%3A4674&styles=&format=application/openlayers


# Função para selecionar geometrias na camada de medidores pelo nome / Não consegue dar zoom...
def sel_med(comboMed):
  cur.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
  FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT ST_Envelope(geom)) As l)) As properties FROM medidor As lg where nome = '%s') As f ) As fc;""" %comboMed)
  json_med_sel = json.dumps(cur.fetchall())
  conn.commit()
  gjs = folium.GeoJson(json_med_sel[2:len(json_med_sel)-2],
                      name = 'Medidor Selecionado')
  gjs.add_to(m)


# Função para selecionar geometrias na camada de edificação pelo local / Não consegue dar zoom...
def sel_edif(comboLocais):
  cur.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
  FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT ST_Envelope(geom)) As l)) As properties FROM edificacao As lg where local = '%s') As f ) As fc;""" %comboLocais)
  json_edif_sel = json.dumps(cur.fetchall())
  conn.commit()
  gjs = folium.GeoJson(json_edif_sel[2:len(json_edif_sel)-2],
                      name = 'Edificações Selecionadas',
                      style_function = lambda feature: style_sel(feature))
  gjs.add_to(m)


# Função seleciona a geometria dos medidores dentro de um campus selecionado e carrega no mapa com simbologia simples de marcador
# Zoom ainda não operacional...
def filtra_medidor(comboLocais):
  cur.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
  FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT id_medidor, nome, link) As l)) As properties FROM medidor As lg, campus_area As ca where ST_Intersects(lg.geom,ca.geom) AND ca.local = '%s') As f ) As fc;""" %comboLocais)
  json_medidor = json.dumps(cur.fetchall())
  conn.commit()
  layer = folium.FeatureGroup(name='Medidores do campus selecionado')
  gj = folium.GeoJson(json_medidor[2:len(json_medidor)-2],
                      name='Medidores do campus selecionado'
                      )

  # iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
  for feature in gj.data['features']:
    if feature['geometry']['type'] == 'Point':
      folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
                    icon = folium.Icon(color = "orange", icon = "star"),
                    ).add_to(layer)
  layer.add_to(m)


# Função para gerar gráfico dinâmico dos medidores
def grafico(comboMed):
  if not (conn is None):
    # Nomes das colunas que indicam o mês/ano de consumo
    lista_periodo = ['c08_2019', 'c09_2019', 'c10_2019', 'c11_2019', 'c12_2019', 
                     'c01_2020', 'c02_2020', 'c03_2020', 'c04_2020', 'c05_2020', 'c06_2020', 'c07_2020', 'c08_2020',
                     'c09_2020', 'c10_2020', 'c11_2020', 'c12_2020', 'c01_2021', 'c02_2021', 'c03_2021', 'c04_2021']
    
    #consulta no BD para gerar um pandas DataFrame a partir da tabela de medidores
    sql = (""" SELECT * FROM medidor where nome = '%s';""" %comboMed)

    df_med = pd.read_sql_query(sql,conn)
    df_med.fillna(0) #trocando valores NaN por 0
    
    data = df_med.iloc[0]
    names = lista_periodo
    values = list(data[lista_periodo])

    fig = plt.figure(figsize = (13,9))
    
    y_pos = np.arange(len(names))
    plt.bar(y_pos,values, align='center', width=0.8)
    plt.xticks(y_pos, names, rotation=45)
    plt.xlabel('Mês/Ano')
    plt.ylabel('Consumo médio mensal (kW/h)')
    plt.title("Consumo médio mensal de energia no medidor")
    st.pyplot(fig)



# -- CÓDIGO PRINCIPAL --- #

def main():
  # este if é para não ir adiante sem a conexão estabelecida
  if not (conn is None):

    # --- CANVA --- #
    #st.title("Projeto Energ! UFPR")

    # Função para carregar bairros no mapa
    bairros()
    # Função para carregar edificações categorizadas no mapa
    edif()

    # Função para carregar todos os medidores no mapa
    #medidor()
    # Função para carregar medidores em círculo concêntrico no mapa / Não está mostrando WMS corretamente
    #med_wms()


    # --- SIDEBAR --- #
    st.sidebar.header('MENU')
    with st.sidebar.expander("Opções de consulta", True):
      #Pega locais e medidores da UFPR para comboBox no widget do streamlit
      comboLocais = st.selectbox('Selecione um campus da UFPR:', lista_local())
      comboMed = st.selectbox('Selecione um medidor para análise:', lista_medidor())

      #Executa a seleção a partir do selectbox
      if comboLocais != '':
        sel_edif(comboLocais) # Função para carregar edificações selecionadas no mapa
        filtra_medidor(comboLocais) # Função para carregar medidores dentro de um campus escolhido
      else:
        pass

      if comboMed != '':
        sel_med(comboMed)  # Função para carregar medidor selecionado no mapa / independente do campus selecionado
      else:
        pass
    
    # Layout da seção inicial
    row1_1, row1_2 = st.columns((1,2))

    IMAGE_URL = "https://energi.eletrica.ufpr.br/static/images/navbar/logo-colorido.svg"

    with row1_1:
      st.title("Projeto Energ! UFPR")
      st.image(IMAGE_URL, width=100)

    with row1_2:
      st.write(
          """
          ##
          Essa é uma aplicação SIG Web no âmbito do Projeto Energ! desenvolvida para divulgação dos dados de consumo de energia elétrica nos Campi da Universidade Federal do Paraná(UFPR),
          no município de Curitiba-PR, permitindo análise espacial e indicação dos índices de consumo registrados nos diferentes medidores da Universidade.
          """)

    #Layout da janela principal
    row2_1, row3_1 = st.columns((2,1))
    with row2_1:
      st.write("")
      st.write("**Local da UFPR selecionado: %s**" % (comboLocais))
      
      folium.LayerControl(collapsed = True).add_to(m)

      #passa o folium para o streamlit
      folium_static(m, width = width, height = height)
      st.write("Base de dados: Projeto UFPR CampusMap e Projeto Energ! (2021)")

    #------- Carrega Gráfico de Barras do Medidor Selecionado -------
    with row3_1:
      st.write("")
      st.write("**Gráfico de barras do consumo médio mensal no medidor: %s**" % (comboMed))
      if comboMed != '':
        grafico(comboMed)  # Função para carregar gráfico de consumo do medidor selecionado no mapa
        st.write("Amplie o gráfico para melhor visualização")
      else:
        pass
     
    
# --- PÁGINA --- #
# Configurações da aba da página
img_icon = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Light-Bulb_icon_by_Till_Teenck_green.svg/120px-Light-Bulb_icon_by_Till_Teenck_green.svg.png"

PAGE_CONFIG = {"page_title":"Projeto Energ! UFPR","page_icon":img_icon,"layout":"wide", "initial_sidebar_state":"expanded"}
st.set_page_config(**PAGE_CONFIG)

if __name__ == '__main__': 
  main()

cur.close()
conn.close()

Writing energi.py


In [ ]:
 !streamlit run --server.port 80 energi.py &>/dev/null&

In [96]:
from pyngrok import ngrok
!ngrok authtoken 1ypdZhRjqFqj7wK66gwA1fXIT6s_WwGc2wpbbDUYCazWgZsB
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


<NgrokTunnel: "http://0c84-35-193-203-110.ngrok.io" -> "http://localhost:80">

In [94]:
# mata o processso do ngork para reiniciar (se necessário)
from pyngrok import ngrok
ngrok.kill()